In [1]:
import copy
import pandas as pd
import numpy as np

df_cases: pd.DataFrame = pd.read_csv('../data/cleaned_csv/2-question_results/cases.csv', sep=',')
df_food : pd.DataFrame = pd.read_csv('../data/cleaned_csv/2-question_results/food.csv', sep=',')

In [2]:
df_cases

,case_id,year,month,state,location,species,status,illnesses,hospitalizations,fatalities
0,1,1998,January,California,Restaurant,NaN,NaN,20,0.0,0.0
1,2,1998,January,California,NaN,NaN,NaN,112,0.0,0.0
2,3,1998,January,California,Restaurant,NaN,NaN,35,0.0,0.0
3,4,1998,January,California,Restaurant,Scombroid toxin,Confirmed,4,0.0,0.0
4,5,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,26,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
17390,17391,2015,December,Wisconsin,Restaurant,Norovirus genogroup II,Confirmed,4,0.0,0.0
17391,17392,2015,December,Wisconsin,Private Home/Residence,Norovirus genogroup II,Confirmed,16,0.0,0.0
17392,17393,2015,December,Wisconsin,Nursing Home/Assisted Living Facility,Norovirus genogroup II,Confirmed,43,3.0,0.0
17393,17394,2015,December,Wisconsin,Fast Food Restaurant,Norovirus genogroup II,Confirmed,5,0.0,0.0


In [3]:
df_outbreaks_merged:  pd.DataFrame = df_cases.merge(df_food, how="inner", on='case_id')

df_outbreaks_grouped: pd.DataFrame = (df_outbreaks_merged
                                     .groupby(['year', 
                                               'month', 
                                               'state', 
                                               'location', 
                                               'species',
                                               'status',
                                               'food'], as_index=False )
                                     .agg(
                                           illnesses        = ('illnesses', 'sum'), 
                                           hospitalizations = ('hospitalizations', 'sum'), 
                                           fatalities       = ('fatalities', 'sum')       )
)

df_outbreaks_grouped

,year,month,state,location,species,status,food,illnesses,hospitalizations,fatalities
0,1998,April,California,Private Home/Residence,Salmonella enterica,Confirmed,unknown,7,0.0,0.0
1,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,pate,11,0.0,0.0
2,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,unspecified,11,0.0,0.0
3,1998,April,California,Restaurant,Shigella boydii,Confirmed,unknown,72,7.0,0.0
4,1998,April,Colorado,Catering Service,Virus,Suspected,salad,11,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
14398,2015,September,Washington,Restaurant,Salmonella enterica,Confirmed,unknown,2,0.0,0.0
14399,2015,September,Washington DC,Restaurant,Salmonella enterica,Confirmed,unknown,160,9.0,0.0
14400,2015,September,Wisconsin,Religious Facility,Salmonella enterica,Confirmed,unknown,10,0.0,0.0
14401,2015,September,Wisconsin,School/College/University,Campylobacter jejuni,Confirmed,chicken,46,3.0,0.0


In [4]:
months = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]

df_outbreaks_grouped['month'] = pd.Categorical(df_outbreaks_grouped['month'], 
                                               categories=months, 
                                               ordered=True)

df_outbreaks_sorted: pd.DataFrame = df_outbreaks_grouped \
                                            .sort_values(by=['year', 'month'])

In [5]:
df_outbreaks_sorted

,year,month,state,location,species,status,food,illnesses,hospitalizations,fatalities
238,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,eggs,26,3.0,0.0
239,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,lasagna,26,3.0,0.0
240,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,other,26,3.0,0.0
241,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,unspecified,26,3.0,0.0
242,1998,January,California,Restaurant,Salmonella enterica,Confirmed,sandwich,4,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
13935,2015,December,Wisconsin,Catering Service,Norovirus genogroup II,Confirmed,cookies,10,0.0,0.0
13936,2015,December,Wisconsin,Fast Food Restaurant,Norovirus genogroup II,Confirmed,unknown,5,0.0,0.0
13937,2015,December,Wisconsin,Nursing Home/Assisted Living Facility,Norovirus genogroup II,Confirmed,unknown,43,3.0,0.0
13938,2015,December,Wisconsin,Private Home/Residence,Norovirus genogroup II,Confirmed,salsa,16,0.0,0.0


In [12]:
# total_illnesses_by_year_state: pd.DataFrame = df_outbreaks_sorted.groupby(['year','state'], as_index=False)['illnesses'].sum()
# total_illnesses_by_year_state

total_illnesses_by_year_state: pd.DataFrame = df_outbreaks_sorted.groupby(['year','state'])['illnesses'].sum()
total_illnesses_by_year_state

year  state        
1998  Alaska              3
      Arizona            58
      Arkansas            8
      California       1802
      Colorado          769
                       ... 
2015  Washington        410
      Washington DC     172
      West Virginia      31
      Wisconsin         688
      Wyoming             6
Name: illnesses, Length: 795, dtype: int64

In [10]:
# df.apply(lambda row: categorise(row), axis=1)

get_totals = lambda row: (total_illnesses_by_year_state.loc[ (row['year'], row['state']) ] )

df_outbreaks_sorted['total_illnesses_by_year_state'] = df_outbreaks_sorted.apply(func=get_totals, axis='columns')
df_outbreaks_sorted

,year,month,state,location,species,status,food,illnesses,hospitalizations,fatalities,total_illnesses_by_year_state
238,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,eggs,26,3.0,0.0,1802
239,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,lasagna,26,3.0,0.0,1802
240,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,other,26,3.0,0.0,1802
241,1998,January,California,Private Home/Residence,Salmonella enterica,Confirmed,unspecified,26,3.0,0.0,1802
242,1998,January,California,Restaurant,Salmonella enterica,Confirmed,sandwich,4,3.0,0.0,1802
...,...,...,...,...,...,...,...,...,...,...,...
13935,2015,December,Wisconsin,Catering Service,Norovirus genogroup II,Confirmed,cookies,10,0.0,0.0,688
13936,2015,December,Wisconsin,Fast Food Restaurant,Norovirus genogroup II,Confirmed,unknown,5,0.0,0.0,688
13937,2015,December,Wisconsin,Nursing Home/Assisted Living Facility,Norovirus genogroup II,Confirmed,unknown,43,3.0,0.0,688
13938,2015,December,Wisconsin,Private Home/Residence,Norovirus genogroup II,Confirmed,salsa,16,0.0,0.0,688
